# Drop insignificant area from source

In [1]:
import os
import mne
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.stats import multitest as mul

mne.viz.set_3d_options(antialias = False)


In [2]:
subjects_dir = '/net/server/data/Archive/prob_learn/freesurfer'


# Download stc with pvalue

In [3]:
stc_pvalue = mne.read_source_estimate('/net/server/data/Archive/prob_learn/pultsinak/sources_sLoreta/averege_betwen_subj/pos_neg_1500_1900/after_fb_lp_pos_vs_neg_pval_fullfdr_1500_1900', 'fsaverage')



# Plotting Pvalue 

In [73]:
scale = [0.95, 0.99, 1.0]
brain = mne.viz.plot_source_estimates(stc_pvalue, hemi='split', views = ['lat', 'med'], 
                                      time_viewer='auto', clim = dict(kind = 'value', lims = scale))    

In [74]:
stc_pvalue_nofdr = mne.read_source_estimate('/net/server/data/Archive/prob_learn/pultsinak/sources_sLoreta/averege_betwen_subj/pos_neg_1500_1900/after_fb_lp_pos_vs_neg_pval_nofdr_1500_1900', 'fsaverage')
scale = [0.95, 0.99, 1.0]
brain = mne.viz.plot_source_estimates(stc_pvalue_nofdr, hemi='split', views = ['lat', 'med'], 
                                      time_viewer='auto', clim = dict(kind = 'value', lims = scale))    

In [50]:
stc_pvalue.data.shape


(20484, 1)

In [51]:
type(stc_pvalue.data)

numpy.ndarray

In [4]:
stc_pvalue_lh_reshape = np.reshape(stc_pvalue.lh_data, 10242)
stc_pvalue_rh_reshape = np.reshape(stc_pvalue.rh_data, 10242)
print(stc_pvalue_lh_reshape[0])

0.998612


In [53]:
stc_pvalue.vertices[0] # left hemispere

array([    0,     1,     2, ..., 10239, 10240, 10241])

In [6]:
index_lh = []

for i, p in enumerate(stc_pvalue_lh_reshape):
    if p >= 0.95 or p <= -0.95:
        index_lh.append(i)

In [7]:
len(index_lh)

7820

In [8]:
index_rh = []

for i, p in enumerate(stc_pvalue_rh_reshape):
    if p >= 0.95 or p <= -0.95:
        index_rh.append(i)

In [9]:
len(index_rh)

6100

# Download stc with data

In [58]:
stc_data = mne.read_source_estimate('/net/server/data/Archive/prob_learn/pultsinak/sources_sLoreta/averege_betwen_subj/pos_neg_1500_1900/after_fb_lp_pos_vs_neg_mean_beta_1500_1900', 'fsaverage')



In [59]:
stc_data.shape

(20484, 1)

In [60]:
stc_data_lh_reshape = np.reshape(stc_data.lh_data, 10242)
stc_data_rh_reshape = np.reshape(stc_data.rh_data, 10242)

In [61]:
# both hemi
hemisphere = [0,1]
indexes = [index_lh, index_rh]
stc_reshape_hemi = [stc_data_lh_reshape, stc_data_rh_reshape]
for i in hemisphere:
    for j in stc_data.vertices[i]:
        if j not in indexes[i]:
            stc_reshape_hemi[i][j] = 0
        

In [62]:
stc_rh_new = stc_data_rh_reshape.reshape((10242, 1))
stc_rh_new.shape

(10242, 1)

In [63]:
stc_lh_new = stc_data_lh_reshape.reshape((10242, 1))
stc_lh_new.shape

(10242, 1)

In [64]:
stc_data.data[:10242] = stc_lh_new
stc_data.data[10242:] = stc_rh_new

In [65]:
scale = [0.48, 0.7, 0.85]
brain = mne.viz.plot_source_estimates(stc_data, hemi='split', views = ['lat', 'med'], 
                                      time_viewer='auto', clim = dict(kind = 'value', lims = scale))   